# An example for converting brat to BIO

In [ ]:
# we use chia dataset as example
# https://www.nature.com/articles/s41597-020-00620-0

In [ ]:
import os
import json
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from shutil import copyfile
import csv

In [ ]:
from spacy.lang.en import English
nlp = English()
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)

In [ ]:
inputpath = f"chia/chia_with_scope"
outputpath = f"chia/chia_bio"
trainpath = f"chia/trains"
testpath = f"chia/tests"

In [ ]:
inputfiles = set()
for f in os.listdir(inputpath):
    if f.endswith('.ann'):
        inputfiles.add(f.split('.')[0].split('_')[0])
len(inputfiles)

In [ ]:
# list of entity types to retain
select_types = ['Condition', 'Value', 'Drug', 'Procedure', 'Measurement', 'Temporal', \
    'Observation', 'Person', 'Mood', 'Device', 'Pregnancy_considerations']

In [ ]:
# convert Brat format into BIO format
# function for getting entity annotations from the annotation file
def get_annotation_entities(ann_file, select_types=None):
    entities = []
    with open(ann_file, "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith('T'):
                term = line.strip().split('\t')[1].split()
                if (select_types != None) and (term[0] not in select_types): continue
                if int(term[-1]) <= int(term[1]): continue
                entities.append((int(term[1]), int(term[-1]), term[0]))
    return sorted(entities, key=lambda x: (x[0], x[1]))

# function for handling overlap by keeping the entity with largest text span
def remove_overlap_entities(sorted_entities):
    keep_entities = []
    for idx, entity in enumerate(sorted_entities):
        if idx == 0:
            keep_entities.append(entity)
            last_keep = entity
            continue
        if entity[0] < last_keep[1]:
            if entity[1]-entity[0] > last_keep[1]-last_keep[0]:
                last_keep = entity
                keep_entities[-1] = last_keep
        elif entity[0] == last_keep[1]:
            last_keep = (last_keep[0], entity[1], last_keep[-1])
            keep_entities[-1] = last_keep
        else:
            last_keep = entity
            keep_entities.append(entity)
    return keep_entities

# inverse index of entity annotations
def entity_dictionary(keep_entities, txt_file):
    f_ann = {}
    with open(txt_file, "r", encoding="utf-8") as f:
        text = f.readlines()
        if file in ['NCT02348918_exc', 'NCT02348918_inc', 'NCT01735955_exc']:
            text = ' '.join([i.strip() for i in text])
        else:
            text = '  '.join([i.strip() for i in text])
    for entity in keep_entities:
        entity_text = text[entity[0]:entity[1]]
        doc = nlp(entity_text)
        token_starts = [(i, doc[i:].start_char) for i in range(len(doc))]
        term_type = entity[-1]
        term_offset = entity[0]
        for i, token in enumerate(doc):
            ann_offset = token_starts[i][1]+term_offset
            if ann_offset not in f_ann:
                f_ann[ann_offset] = [i, token.text, term_type]
    return f_ann

In [ ]:
# Brat -> BIO format conversion
for infile in inputfiles:
    for t in ["exc", "inc"]:
        file = f"{infile}_{t}"
        ann_file = f"{inputpath}/{file}.ann"
        txt_file = f"{inputpath}/{file}.txt"
        out_file = f"{outputpath}/{file}.bio.txt"
        sorted_entities = get_annotation_entities(ann_file, select_types)
        keep_entities = remove_overlap_entities(sorted_entities)
        f_ann = entity_dictionary(keep_entities, txt_file)
        with open(out_file, "w", encoding="utf-8") as f_out:
            with open(txt_file, "r", encoding="utf-8") as f:
                sent_offset = 0
                for line in f:
                    if '⁄' in line:
                        line = line.replace('⁄', '/') # replace non unicode characters
                    doc = nlp(line.strip())
                    token_starts = [(i, doc[i:].start_char) for i in range(len(doc))]
                    for token in doc:
                        token_sent_offset = token_starts[token.i][1]
                        token_doc_offset = token_starts[token.i][1]+sent_offset
                        if token_doc_offset in f_ann:
                            if f_ann[token_doc_offset][0] == 0:
                                label = f"B-{f_ann[token_doc_offset][2]}"
                            else:
                                label = f"I-{f_ann[token_doc_offset][2]}"
                        else:
                            label = f"O"
                        # print(token.text, token_sent_offset, token_sent_offset+len(token.text), token_doc_offset, token_doc_offset+len(token.text), label)
                        f_out.write(f"{token.text} {token_sent_offset} {token_sent_offset+len(token.text)} {token_doc_offset} {token_doc_offset+len(token.text)} {label}\n")
                    # print('\n')
                    f_out.write('\n')
                    if file in ['NCT02348918_exc', 'NCT02348918_inc', 'NCT01735955_exc']: # 3 trials with inconsistent offsets
                        sent_offset += (len(line.strip())+1)
                    else:
                        sent_offset += (len(line.strip())+2)

In [ ]:
# dataset separation: 800 trials (80%) for training, 100 trials (10%) for validation and 100 trials (10%) for testing
train_ids, dev_ids = train_test_split(list(inputfiles), train_size=0.8, random_state=13, shuffle=True)
dev_ids, test_ids = train_test_split(dev_ids, train_size=0.5, random_state=13, shuffle=True)
print(len(train_ids), len(dev_ids), len(test_ids))
chia_datasets = {"train":train_ids, "dev":dev_ids, "test":test_ids}
json.dump(chia_datasets, open("chia/chia_datasets.json", "w", encoding="utf-8"))

In [ ]:
# Merge BIO format train, validation and test datasets
# merge the train dataset
with open("chia/train.txt", "w", encoding="utf-8") as f:
    for fid in chia_datasets["train"]:
        copyfile(f"{outputpath}/{fid}_exc.bio.txt", f"{trainpath}/{fid}_exc.bio.txt")
        copyfile(f"{outputpath}/{fid}_inc.bio.txt", f"{trainpath}/{fid}_inc.bio.txt")
        with open(f"{outputpath}/{fid}_exc.bio.txt", "r", encoding="utf-8") as fr:
            txt = fr.read().strip()
            if txt != '':
                f.write(txt)
                f.write("\n\n")
        with open(f"{outputpath}/{fid}_inc.bio.txt", "r", encoding="utf-8") as fr:
            txt = fr.read().strip()
            if txt != '':
                f.write(txt)
                f.write("\n\n")

# merge the validation dataset
with open("chia/dev.txt", "w", encoding="utf-8") as f:
    for fid in chia_datasets["dev"]:
        copyfile(f"{outputpath}/{fid}_exc.bio.txt", f"{trainpath}/{fid}_exc.bio.txt")
        copyfile(f"{outputpath}/{fid}_inc.bio.txt", f"{trainpath}/{fid}_inc.bio.txt")
        with open(f"{outputpath}/{fid}_exc.bio.txt", "r", encoding="utf-8") as fr:
            txt = fr.read().strip()
            if txt != '':
                f.write(txt)
                f.write("\n\n")
        with open(f"{outputpath}/{fid}_inc.bio.txt", "r", encoding="utf-8") as fr:
            txt = fr.read().strip()
            if txt != '':
                f.write(txt)
                f.write("\n\n")

# merge the test dataset
with open("chia/test.txt", "w", encoding="utf-8") as f:
    for fid in chia_datasets["test"]:
        copyfile(f"{outputpath}/{fid}_exc.bio.txt", f"{testpath}/{fid}_exc.bio.txt")
        copyfile(f"{outputpath}/{fid}_inc.bio.txt", f"{testpath}/{fid}_inc.bio.txt")
        with open(f"{outputpath}/{fid}_exc.bio.txt", "r", encoding="utf-8") as fr:
            txt = fr.read().strip()
            if txt != '':
                f.write(txt)
                f.write("\n\n")
        with open(f"{outputpath}/{fid}_inc.bio.txt", "r", encoding="utf-8") as fr:
            txt = fr.read().strip()
            if txt != '':
                f.write(txt)
                f.write("\n\n")